# Fit the diff color model for PWV

- author Sylvie Dagoret-Campagne
- creation date : 2024/09/10
- update : 2024/08/12

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os,sys
import re
import pandas as pd

from astropy.io import fits
from astropy import units as u
from astropy import constants as c

plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import pickle
from scipy.interpolate import RegularGridInterpolator

import scipy
from scipy.optimize import curve_fit

In [ ]:
machine_name = os.uname().nodename
path_rubinsimphot = "repos/repos_w_2024_17/rubinsimphot/src"
if 'sdf' in machine_name:
    #machine_name_usdf = 'sdfrome001'
    print("Set environment for USDF")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)
elif 'dagoret-nb' in machine_name:
    print("Set environment for USDF Rubin Science Platform")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)    
elif 'mac' in machine_name:
    print("Be sure to run this notebook in conda environment named conda_py310")
else:
    print(f"Your current machine name is {machine_name}. Check your python environment")

## Config

In [ ]:
OBS_tag = "AUXTEL"
file0_out = f"{OBS_tag}_atmosphericzycolorgrid_params.pickle"
file1_out = f"{OBS_tag}_atmosphericzycolorgrid_datagrid.npy"

## Read files

In [ ]:
with (open(file0_out, "rb")) as openfile:
    while True:
        try:
            info_params = pickle.load(openfile)
        except EOFError:
            break


In [ ]:
info_params

In [ ]:
all_pwv = info_params['PWV']
all_zycolordiff = info_params['ZYCOLDIFF']

In [ ]:
datagrid = np.load(file1_out,mmap_mode=None, allow_pickle=False)
print(f"...... datagrid file {file1_out} read")

## Check the model

In [ ]:
nx = len(all_pwv)
ny = len(all_zycolordiff)
xg, yg = np.meshgrid(all_pwv, all_zycolordiff , indexing='ij')

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(3,1,figsize=(8,10))
im = ax1.imshow(datagrid,origin="lower",cmap="jet",
           extent=(all_pwv.min(),all_pwv.max(),all_zycolordiff.min(),all_zycolordiff.max()),aspect='auto')
#cbar

ax1.set_title("relative color difference : $(Z-Y)_1 - (Z-Y)_2$ ")
ax1.set_ylabel("$(Z-Y)_1-(Z-Y)_2$")
ax1.set_xlabel("pwv (mm)")
#ax1.set_aspect('auto')

ax2.plot(all_pwv, datagrid[0,:],'b-')
ax2.set_xlabel("pwv (mm)")
ax2.set_ylabel("rel. col. : $(Z-Y)_1 - (Z-Y)_2$")
ax3.plot(all_pwv, datagrid[-1,:],'r-')
ax3.set_xlabel("pwv (mm)")
ax3.set_ylabel("rel. col. : $(Z-Y)_1 - (Z-Y)_2$")

plt.tight_layout()
plt.show()

In [ ]:
interp = RegularGridInterpolator((all_pwv, all_zycolordiff), datagrid.T, bounds_error=False, fill_value=None)

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=all_zycolordiff.min(), vmax=all_zycolordiff.max())
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,6))

for z_y0 in all_zycolordiff:
    the_color = scalarMap.to_rgba(z_y0, alpha=1)
    
    pts = list(zip(all_pwv,z_y0*np.ones(len(all_pwv))))  
    c = interp(pts)
    ax.plot(all_pwv,c,'-',c=the_color)
    
cbar = fig.colorbar(scalarMap,ax=ax, orientation='vertical', label='Z-Y(pwv=0) (mag)')    
ax.set_xlabel("pwv (mm)")    
#ax.set_ylabel("Czy1_Czy2_abmag_rel (mmag)")
ax.set_ylabel("rel. col. : $(Z-Y)_1 - (Z-Y)_2$ (mmag)")
ax.set_title("Relative color difference Z-Y from star1,star2")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,6))

for idx,z_y0 in enumerate(all_zycolordiff):
    the_color = scalarMap.to_rgba(z_y0, alpha=1)
    
    pts = list(zip(all_pwv,z_y0*np.ones(len(all_pwv))))  
    c = interp(pts)
    ax.plot(all_pwv,c,'o',lw=0.5,c=the_color)
    
    #p = np.polyfit(np.log(np.array(all_pwv)), np.log(np.abs(c)),deg=1) 
    #print(idx,z_y0,p)
    
cbar = fig.colorbar(scalarMap,ax=ax, orientation='vertical', label='Z-Y(pwv=0) (mag)')    
ax.set_xlabel("pwv (mm)")    
#ax.set_ylabel("Czy1_Czy2_abmag_rel (mmag)")
ax.set_ylabel("rel. col. : $(Z-Y)_1 - (Z-Y)_2$ (mmag)")
ax.set_title("Relative color difference Z-Y from star1,star2")
ax.set_yscale('log')
ax.set_xscale('log')
ax.grid()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,6))

all_p = []
all_sel_dc = []

for idx,z_y0 in enumerate(all_zycolordiff):
    the_color = scalarMap.to_rgba(z_y0, alpha=1)
    
    pts = list(zip(all_pwv,z_y0*np.ones(len(all_pwv))))  
    c = interp(pts)
    
    X = np.log(all_pwv[1:])
    
    csum = c[1:].sum()
    
    
    if csum>0 :
        Y = np.log(c[1:])
    elif csum<0 :
        Y = - np.log(-c[1:])
    else:
        Y = 0.0
        
    if csum != 0.0:    
        ax.plot(X,Y,'.',lw=0.5,c=the_color)
    
        z = np.polyfit(X,Y,deg=1) 
        pol = np.poly1d(z)
    
        xfit = np.linspace(-2,2,50)
        yfit = pol(xfit)
    
        plt.plot(xfit,yfit,'-',c=the_color)
        all_p.append(z)
        all_sel_dc.append(z_y0)
    
cbar = fig.colorbar(scalarMap,ax=ax, orientation='vertical', label='Z-Y(pwv=0) (mag)')    
ax.set_xlabel("log(pwv (mm))")    
#ax.set_ylabel("Czy1_Czy2_abmag_rel (mmag)")
ax.set_ylabel("log of rel. col. : $(Z-Y)_1 - (Z-Y)_2$ (mmag)")
ax.set_title("Fit model polynom deg 1 : log(DC) = log(DC0) + beta(log(PWV)) ")

ax.grid()

all_sel_dc = np.array(all_sel_dc)

In [ ]:
all_sel_dc = np.reshape(all_sel_dc, (-1, 1))
all_sel_dc

In [ ]:
all_p = np.vstack(all_p)
all_p.shape

In [ ]:
fit_res = np.hstack([all_sel_dc,all_p])
fit_res

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,4))
ax1.plot(fit_res[:,0],fit_res[:,2],'bo')
ax1.set_xlabel("Z-Y0 (mag)")
ax1.set_ylabel("constant")
ax2.plot(fit_res[:,0],fit_res[:,1],'go')
ax2.set_xlabel("Z-Y0 (mag)")
ax2.set_ylabel("slope (exponent of pwv)")
plt.suptitle("diff color fit : polynom 1d")
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,6))

all_p = []
all_sel_dc = []

for idx,z_y0 in enumerate(all_zycolordiff):
    the_color = scalarMap.to_rgba(z_y0, alpha=1)
    
    pts = list(zip(all_pwv,z_y0*np.ones(len(all_pwv))))  
    c = interp(pts)
    
    X = np.log(all_pwv[1:])
    csum = c[1:].sum()
    
    if csum>0 :
        Y = np.log(c[1:])
    elif csum<0 :
        Y = - np.log(-c[1:])
    else:
        Y = 0.0
        
    if csum != 0.0:    
        ax.plot(X,Y,'.',lw=0.5,c=the_color)
    
        z = np.polyfit(X,Y,deg=2) 
        pol = np.poly1d(z)
    
        xfit = np.linspace(-2,2,50)
        yfit = pol(xfit)
    
        plt.plot(xfit,yfit,'-',c=the_color)
        all_p.append(z)
        all_sel_dc.append(z_y0)
    
    

cbar = fig.colorbar(scalarMap,ax=ax, orientation='vertical', label='Z-Y(pwv=0) (mag)')    
ax.set_xlabel("log(pwv (mm))")    
#ax.set_ylabel("Czy1_Czy2_abmag_rel (mmag)")
ax.set_ylabel("log(rel. col. : $(Z-Y)_1 - (Z-Y)_2$ (mmag))")
ax.set_title("Fit model polynom deg 2 : log(DC) = log(DC0) + beta(log(PWV)) + beta(log(PWV)^2)")

ax.grid()
all_sel_dc = np.array(all_sel_dc)

In [ ]:
all_sel_dc = np.reshape(all_sel_dc, (-1, 1))
all_sel_dc

In [ ]:
all_p = np.vstack(all_p)
all_p.shape

In [ ]:
fit_res = np.hstack([all_sel_dc,all_p])
fit_res

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(15,4))
ax1.plot(fit_res[:,0],fit_res[:,3],'ro')
ax1.set_xlabel("Z-Y0 (mag)")
ax1.set_ylabel("constant")
ax2.plot(fit_res[:,0],fit_res[:,2],'bo')
ax2.set_xlabel("Z-Y0 (mag)")
ax2.set_ylabel("slope (exponent of pwv")
ax3.plot(fit_res[:,0],fit_res[:,1],'go')
ax3.set_xlabel("Z-Y0 (mag)")
ax3.set_ylabel("")
plt.suptitle("diff color fit : polynom deg=2")
plt.tight_layout()